# Minneapolis, MN Simulations
8760 rows per simulation<br>
196 files (removed cell-in-cell)

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [2]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 0.83
elec = 0.1314

---

## Combine CSV Files in Directory

In [3]:
# Create variable for files in directory
files = [f for f in os.listdir("data/msp/") if f.endswith(".csv")]

# files

In [4]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/msp/" + file)
    all_data = pd.concat([all_data, df])
    
#all_data

# 1716960  rows, 73 columns

In [5]:
# all_data.dtypes

In [6]:
# print(all_data)
# is_NaN = all_data. isnull()
# row_has_NaN = is_NaN. any(axis=1)
# rows_with_NaN = all_data[row_has_NaN]
# print(rows_with_NaN)

In [7]:
# Get wanted columns
all_data1 = all_data[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "]]

#all_data1

In [8]:
all_data1 = all_data1.dropna().reset_index(drop=True)

#all_data1

#1716960  rows, 73 columns

-----

## Get the Wanted Data (columns)

In [9]:
# Rename Columns
all_data2 = all_data1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data2

#1716960  rows, 6 columns

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [10]:
# Copy dataframe to modify and leave original df intact
get_annual = all_data2.copy(deep=True)

#get_annual

In [11]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual

In [12]:
# Drop columns
get_annual.drop(columns = ["Illuminance(lux)", "FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual

---

## Get Annual Cost Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>
Add cost multiplier, variables defined above

In [13]:
# Get annual costs
get_annual_costs = all_data2.copy(deep=True)

In [14]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanEnergy($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual_costs["HeatingEnergy($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual_costs["CoolingEnergy($)"] = (get_annual_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs

In [15]:
# Drop columns
get_annual_costs.drop(columns = ["Illuminance(lux)", "FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual_costs

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Add baseline data <br>
Export combine file

In [16]:
# Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs)

#allAnnualData

In [17]:
# Add energy and cost totals
totalEnergy = allAnnualData["FanEnergy(kBtu)"] + allAnnualData["HeatingEnergy(kBtu)"] + allAnnualData["CoolingEnergy(kBtu)"]
allAnnualData["AnnualEnergy(kBtu)"] = totalEnergy

totalCost = allAnnualData["FanEnergy($)"] + allAnnualData["HeatingEnergy($)"] + allAnnualData["CoolingEnergy($)"]
allAnnualData["AnnualCost($)"] = totalCost

#allAnnualData


### Establish Annual Energy and Cost Baseline Data


In [18]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_annual = allAnnualData[allAnnualData["Scenario"] == "Clear/Clear-VB0"]

#baseline_annual

In [19]:
annualEnergy_fan = baseline_annual["FanEnergy(kBtu)"].sum()
annualEnergy_fan

2518.5942054690404

In [20]:
annualEnergy_cool = baseline_annual["CoolingEnergy(kBtu)"].sum()
annualEnergy_cool

9148.732496988963

In [21]:
annualEnergy_heat = baseline_annual["HeatingEnergy(kBtu)"].sum()
annualEnergy_heat

106490.68918688194

In [22]:
annualEnergy = (annualEnergy_fan + annualEnergy_cool + annualEnergy_heat)
annualEnergy

118158.01588933993

In [23]:
annualCost_fan = baseline_annual["FanEnergy($)"].sum()
annualCost_fan

96.98990093485853

In [24]:
annualCost_cool = baseline_annual["CoolingEnergy($)"].sum()
annualCost_cool

352.31346782092317

In [25]:
annualCost_heat = baseline_annual["HeatingEnergy($)"].sum()
annualCost_heat

883.87272025112

In [26]:
annualCost = (annualCost_fan + annualCost_cool + annualCost_heat)
annualCost

1333.1760890069017

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Add baselines to dataframe<br>
Export to file

In [27]:
# Group by scenario
annualData = allAnnualData.groupby(["Scenario"], as_index=False)

#annualData.sum().round(2)

In [28]:
annualDataFinal = annualData.sum().round(2)

#annualDataFinal

In [29]:
# Add columns for delta from base
annualDataFinal["DeltaEnergyFromBase"] = annualDataFinal["AnnualEnergy(kBtu)"] - annualEnergy
annualDataFinal["DeltaCostFromBase"] = annualDataFinal["AnnualCost($)"] - annualCost

#annualDataFinal.round(2)

In [30]:
# Export grouped file to csv
annualDataFinal.to_csv("../Scraper_Output/msp_annual_data.csv", header=True, index=True)

-----

## Get Monthly Energy Data
Change units from Joules to therms and kWh<br>
Drop unwanted columns

In [31]:
# Copy dataframe to modify and leave original df intact
get_monthly = all_data2.copy(deep=True)

#get_monthly

In [32]:
# Convert Joules to kWh and therms in new dataframe
get_monthly["FanEnergy(kWh)"] = get_monthly["FanEnergy[J](Hourly)"] * .000000277777778
get_monthly["HeatingEnergy(therms)"] = get_monthly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_monthly["CoolingEnergy(kWh)"] = get_monthly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_monthly

In [33]:
# Drop columns
get_monthly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly

In [34]:
# get_monthly["Month"] = get_monthly["Date_Time"].apply(lambda x: str(x)[0:3])

# get_monthly

## Get Monthly Cost Data

Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [35]:
# Copy dataframe to modify and leave original df intact
get_monthly_costs = all_data2.copy(deep=True)

#get_monthly

In [36]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly_costs["FanEnergy($)"] = (get_monthly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly_costs["HeatingEnergy($)"] = ((get_monthly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly_costs["CoolingEnergy($)"] = (get_monthly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [37]:
# Drop columns
get_monthly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly_costs

In [38]:
# get_monthly_costs["Month"] = get_monthly_costs["Date_Time"].apply(lambda x: str(x)[0:3])

# get_monthly_costs

---

## Combine Monthly Energy and Cost Data
Combine two dfs<br>
Create Month Column<br>
Establish baseline <br>
Export combine file

In [39]:
# Combine data for energy and cost
allMonthlyData = pd.merge(get_monthly, get_monthly_costs, how="inner" )

#allMonthlyData

In [40]:
# Add energy and cost totals
# totalEnergyMonthly = allMonthlyData["FanEnergy(kWh)"] + allMonthlyData["HeatingEnergy(therms)"] + allMonthlyData["CoolingEnergy(kWh)"]
# allMonthlyData["MonthlyEnergy(kBtu)"] = totalEnergyMonthly

totalCostMonthly = allMonthlyData["FanEnergy($)"] + allMonthlyData["HeatingEnergy($)"] + allMonthlyData["CoolingEnergy($)"]
allMonthlyData["MonthlyCost($)"] = totalCostMonthly

averageIlluminanceMonthly = allMonthlyData["Illuminance(lux)"].mean()

#allMonthlyData

In [41]:
allMonthlyData["Month"] = allMonthlyData.Date_Time.str.slice(0,3)

#allMonthlyData

In [42]:
# Drop Illuminance column...don't need the totals
allMonthlyData2 = allMonthlyData.drop(["Illuminance(lux)"], axis=1)
#allMonthlyData2


### Establish Monthly Energy and Cost Baseline Data


In [43]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_monthly = allMonthlyData2[allMonthlyData2["Scenario"] == "Clear/Clear-VB0"]

#baseline_monthly

In [44]:
monthlyEnergy_fan = baseline_monthly["FanEnergy(kWh)"].sum()
monthlyEnergy_fan

738.1270999608715

In [45]:
monthlyEnergy_cool = baseline_monthly["CoolingEnergy(kWh)"].sum()
monthlyEnergy_cool

2681.228826643251

In [46]:
monthlyEnergy_heat = baseline_monthly["HeatingEnergy(therms)"].sum()
monthlyEnergy_heat

1064.9068918688192

In [47]:
monthlyCost_fan = baseline_monthly["FanEnergy($)"].sum()
monthlyCost_fan

96.98990093485853

In [48]:
monthlyCost_cool = baseline_monthly["CoolingEnergy($)"].sum()
monthlyCost_cool

352.31346782092317

In [49]:
monthlyCost_heat = baseline_monthly["HeatingEnergy($)"].sum()
monthlyCost_heat

883.87272025112

In [50]:
monthlyCost = (monthlyCost_fan + monthlyCost_cool + monthlyCost_heat)
monthlyCost

1333.1760890069017

## Group and Export Monthly Data
Group dataframe by scenario and month<br>
Add baselines to dataframe<br>
Export to file

In [51]:
# Group by scenario and date/time for export to graphing
MonthlyData = allMonthlyData2.groupby(["Scenario", "Month"], as_index=False)

#MonthlyData.sum()

In [52]:
MonthlyDataFinal = MonthlyData.sum()

#MonthlyDataFinal

In [53]:
# Add columns for delta from base
MonthlyDataFinal["DeltaFanFromBase"] = MonthlyDataFinal["FanEnergy(kWh)"] - monthlyEnergy_fan
MonthlyDataFinal["DeltaCoolFromBase"] = MonthlyDataFinal["CoolingEnergy(kWh)"] - monthlyEnergy_cool
MonthlyDataFinal["DeltaHeatFromBase"] = MonthlyDataFinal["HeatingEnergy(therms)"] - monthlyEnergy_heat
MonthlyDataFinal["DeltaCostFromBase"] = MonthlyDataFinal["MonthlyCost($)"] - monthlyCost

#MonthlyDataFinal.round(2)

In [54]:
# Export grouped file to csv (used before grouped by month)
MonthlyDataFinal.round(2).to_csv("../Scraper_Output/msp_monthly_data.csv", header=True, index=True)

In [55]:
# # Group by scenario and date/time for export to graphing
# MonthlyData2Final = allMonthlyData2.groupby(["Scenario", "Month"], as_index=False)

# MonthlyData2Final.sum()

In [56]:
# # Export grouped file to csv
# MonthlyData2Final.sum().round(2).to_csv("../Scraper_Output/msp_monthly2_data.csv", header=True, index=True)

## Get Monthly Average Illuminance
Get just the illuminance column<br>
Group by scenario <br>
Groups by month <br>

In [57]:
Illuminance = allMonthlyData[["Scenario", "Date_Time", "Illuminance(lux)", "Month"]]

#Illuminance

In [58]:
# Group by scenario
MonthlyIlluminance = Illuminance.groupby(["Scenario", "Month"])

#MonthlyIlluminance.mean()

In [59]:
# Export grouped file to csv
MonthlyIlluminance.mean().round(2).to_csv("../Scraper_Output/msp_monthly_illuminance.csv", header=True, index=True)

------

## Get Hourly Energy Data
Change units from Joules to therms and kWh<br>
Drop unwanted columns

In [60]:
# Copy dataframe to modify and leave original df intact
all_data3 = all_data1.copy(deep=True)

#all_data3

In [61]:
# Get wanted columns
# Include hourly illumance
all_data3 = all_data3[["Scenario",
               "Date/Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "
                      ]]

#all_data3

In [62]:
# Rename Columns
get_hourly = all_data3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly

In [63]:
# Convert Joules to kWh and therms in new dataframe
get_hourly["FanEnergy(kWh)"] = get_hourly["FanEnergy[J](Hourly)"] * .000000277777778
get_hourly["HeatingEnergy(therms)"] = get_hourly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_hourly["CoolingEnergy(kWh)"] = get_hourly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_hourly

In [64]:
# Drop columns
get_hourly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly

------

## Get Hourly Cost Data
Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [65]:
# Copy dataframe to modify and leave original df intact
get_hourly_costs = all_data3.copy(deep=True)

#get_hourly_costs

In [66]:
# Rename Columns
get_hourly_costs = get_hourly_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

# get_hourly_costs

In [67]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly_costs["FanEnergy($)"] = (get_hourly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly_costs["HeatingEnergy($)"] = ((get_hourly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly_costs["CoolingEnergy($)"] = (get_hourly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [68]:
# Drop columns
get_hourly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly_costs

## Combine Hourly Energy and Cost Data
Combine two dfs<br>
Establish baseline <br>
Export combine file

In [69]:
# Combine data for energy and cost
allHourlyData = pd.merge(get_hourly, get_hourly_costs, how="inner" )

#allHourlyData

In [70]:
# Add energy cost totals

totalCosthourly = allHourlyData["FanEnergy($)"] + allHourlyData["HeatingEnergy($)"] + allHourlyData["CoolingEnergy($)"]
allHourlyData["HourlyCost($)"] = totalCosthourly

#allHourlyData


### Establish Hourly Energy and Cost Baseline Data


In [71]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_hourly = allHourlyData[allHourlyData["Scenario"] == "Clear/Clear-VB0"]

#baseline_hourly

In [72]:
hourlyEnergy_fan = baseline_hourly["FanEnergy(kWh)"].mean()
hourlyEnergy_fan

0.08426108447041908

In [73]:
hourlyEnergy_cool = baseline_hourly["CoolingEnergy(kWh)"].mean()
hourlyEnergy_cool

0.3060763500734302

In [74]:
hourlyEnergy_heat = baseline_hourly["HeatingEnergy(therms)"].mean()
hourlyEnergy_heat

0.12156471368365519

In [75]:
hourlyCost_fan = baseline_hourly["FanEnergy($)"].mean()
hourlyCost_fan

0.011071906499413016

In [76]:
hourlyCost_cool = baseline_hourly["CoolingEnergy($)"].mean()
hourlyCost_cool

0.04021843239964872

In [77]:
hourlyCost_heat = baseline_hourly["HeatingEnergy($)"].mean()
hourlyCost_heat

0.10089871235743367

In [78]:
hourlyCost = (hourlyCost_fan + hourlyCost_cool + hourlyCost_heat)
hourlyCost

0.1521890512564954

## Export Hourly Data

Add baselines to dataframe<br>
Export to file

In [79]:
# Add columns for delta from base
allHourlyData["DeltaFanFromBase"] = allHourlyData["FanEnergy(kWh)"] - hourlyEnergy_fan
allHourlyData["DeltaCoolFromBase"] = allHourlyData["CoolingEnergy(kWh)"] - hourlyEnergy_cool
allHourlyData["DeltaHeatFromBase"] = allHourlyData["HeatingEnergy(therms)"] - hourlyEnergy_heat
allHourlyData["DeltaCostFromBase"] = allHourlyData["HourlyCost($)"] - hourlyCost

allHourlyData.round(2)

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),HourlyCost($),DeltaFanFromBase,DeltaCoolFromBase,DeltaHeatFromBase,DeltaCostFromBase
0,Clear/lowE(3)-VB-15,01/01 01:00:00,0.0,0.12,0.26,0.0,0.02,0.22,0.0,0.23,0.04,-0.31,0.14,0.08
1,Clear/lowE(3)-VB-15,01/01 02:00:00,0.0,0.13,0.30,0.0,0.02,0.25,0.0,0.26,0.05,-0.31,0.17,0.11
2,Clear/lowE(3)-VB-15,01/01 03:00:00,0.0,0.14,0.30,0.0,0.02,0.25,0.0,0.27,0.05,-0.31,0.18,0.12
3,Clear/lowE(3)-VB-15,01/01 04:00:00,0.0,0.14,0.30,0.0,0.02,0.25,0.0,0.27,0.05,-0.31,0.18,0.12
4,Clear/lowE(3)-VB-15,01/01 05:00:00,0.0,0.14,0.31,0.0,0.02,0.26,0.0,0.27,0.06,-0.31,0.19,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1733323,lowE(2)/lowE(4)-BBG75,12/31 20:00:00,0.0,0.08,0.18,0.0,0.01,0.15,0.0,0.16,-0.00,-0.31,0.06,0.01
1733324,lowE(2)/lowE(4)-BBG75,12/31 21:00:00,0.0,0.08,0.18,0.0,0.01,0.15,0.0,0.16,-0.00,-0.31,0.06,0.01
1733325,lowE(2)/lowE(4)-BBG75,12/31 22:00:00,0.0,0.08,0.18,0.0,0.01,0.15,0.0,0.16,-0.00,-0.31,0.06,0.01
1733326,lowE(2)/lowE(4)-BBG75,12/31 23:00:00,0.0,0.09,0.19,0.0,0.01,0.16,0.0,0.17,0.00,-0.31,0.07,0.02


In [80]:
# Export grouped file to csv for daily cost sums
allHourlyData.to_csv("../Scraper_Output/msp_daily.csv", header=True, index=False)